# this notebook creates the `rgitools/docs/dems_subregions.rst` file for rgitools docs and copies the new barplot and csv files to their corresponding directories

**NOTE:** 
- **there is a `README.rst` in this notebook's directory which gives instructions on the preceding steps necessary to execute this notebook**
- **this notebook has to be executed in the directory `rgitools/notebooks/dem_statistics` in order to handle repository intern paths correctly and to import the `statistics_paths.py` script which is located in the same directory.**
- **the paths `statistics_paths.py` have to be adapted to your local system's paths and have to remain unchanged during the execution of the `post_all_dems.py` and the `dem_post_quality_per_region.py`**

- **The (old) barplot and csv files in the directories `rgitools/docs/_static/images/` and `rgitools/docs/_static/images/` that might be outdated and unsued after the run of this script have to be deleted manually.**


In [ ]:
import os
import os.path as path
import shutil
import geopandas as gpd
import numpy as np
from oggm import cfg


import statistics_paths
cfg.initialize()

This notebook can be run as a script with parameters using [papermill](https://github.com/nteract/papermill), but it is not necessary nor intended, as the parameters are taken from the `statistics_paths.py` script. The following cell contains the parameters you can choose from:

In [ ]:
# path to the postprocessing output directory which contains the barplots and csv-tables within its subdirectories
post = None
# suffix of the filenames
sfx = None

In [ ]:
if not post:
    post = statistics_paths.post
if not sfx:
    sfx = statistics_paths.sfx

## copy barplot images and csv files

In [ ]:
barplot_source_dir = path.join(post, 'out/images')
csv_source_dir = path.join(post, 'out/tables')

barplot_target_dir = path.abspath(path.join(os.getcwd(), '../../docs/_static/images'))
csv_target_dir = path.abspath(path.join(os.getcwd(), '../../docs/_static/tables'))

# you can simply delete the folder 'images' and 'tables' before copying the file, so you don't have to worry about outdated files
os.makedirs(barplot_target_dir, exist_ok=True)
os.makedirs(csv_target_dir, exist_ok=True)

In [ ]:
for filename in os.listdir(barplot_source_dir):
    shutil.copy(path.join(barplot_source_dir, filename), barplot_target_dir)
print('copied all barplots from {} to {}'.format(barplot_source_dir, barplot_target_dir))
for filename in os.listdir(csv_source_dir):
    shutil.copy(path.join(csv_source_dir, filename), csv_target_dir)
print('copied all csv files from {} to {}'.format(csv_source_dir, csv_target_dir))

## create `dems_subregions.rst` rst file

In [ ]:
# path to the file whose content is created in this script and will be overwritten in the end
subregion_rts_filepath = path.abspath(path.join(os.getcwd(), '../../docs/dems_subregions.rst'))

# rgi region & subregion file - depending on the RGI version (6.0, 6.x ..) chosen for the RGI Topo Dataset creation
# this folder- and filenames might be adapted
regions = gpd.read_file(os.path.join(cfg.PATHS['rgi_dir'], 'RGIV62',
                                     '00_rgi62_regions',
                                     '00_rgi62_O1Regions.shp'))


In [ ]:
# static strings
page_header = '.. _subregions:\n\nDetails for RGI subregions\n==========================\n\n'
table_info = ('Summary of all the RGI subregions. First column shows total\n    '
              'number of glaciers per RGI subregion. The consecutive columns specify the\n    '
              'availability of particular DEMs for a RGI subregion in percent of the total\n    '
              'glaciers per subregion. Values are not rounded but truncated so 99% could\n    '
              'be just one missing glacier. Only DEMs with less than 10% missing values\n    '
              'are considered.')
# path strings
img_dir = '/_static/images/'
csv_dir = '_static/tables/'              

In [ ]:
string_list = [page_header]
    
for reg in np.arange(1,20):
    # region string
    rs = []
    
    regstr = '{:02.0f}'.format(reg)
    regname = regions.loc[regions['RGI_CODE'].astype('int') == reg, 'FULL_NAME'].iloc[0]
    barplot_name = 'barplot_rgi{}.png'.format(regstr + sfx)
    csv_name = 'dem_rgi{}.csv'.format(regstr + sfx)
    
    # region header
    rs.append('.. _rgi{}:\n\n{} - {}\n'.format(regstr, regstr, regname))
    
    # header_underscore with the same length as region number and name
    rs.append(('-' * len('{} - {}'.format(regstr, regname))) + '\n\n')
    
    # barplot
    rs.append('.. image:: {}\n    :width: 100%\n\n'.format(path.join(img_dir, barplot_name)))
    
    # table
    rs.append('.. csv-table:: {}\n    :file: {}\n\n\n'.format(table_info, path.join(csv_dir, csv_name)))
    
    string_list.append(''.join(rs))
    
# remove empty lines at file ending
string_list[-1] = string_list[-1][:-3]

## The following cell will *overwrite(!)* the `dems_subregions.rst` file

In [ ]:
# write the .rst file
subreg_rst_file = open(subregion_rts_filepath, 'w')
subreg_rst_file.write(''.join(string_list))
subreg_rst_file.close()
print('wrote new \'rgitools/docs/dems_subregions.rst\' file\ncheck the new file for correctness')